# Incorporating live orders into your holdings view

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders and allocations. We can show updated positions that account for these live, partially allocated orders.

To illustrate, below we will trade <i>Amazon, Inc.</i>. We'll create a set of limit, stop, and market orders, and show an updated view of the portfolio intraday.


In [ ]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import json
import random
from datetime import datetime, timezone
from IPython.core.display import HTML
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.lpt.lpt import to_date
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
portfolios_api = api_factory.build(lu.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.TransactionPortfoliosApi)
orders_api = api_factory.build(lu.OrdersApi)
allocations_api = api_factory.build(lu.AllocationsApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)

Define scope, portfolio and some other variables used in the example:

In [ ]:
scope = "holdings_with_live_orders"
portfolio_code = "us_long"
strategy_shk = "strategy"
date_today=datetime.today
pf_created_date = "2020-01-01T00:00:00+00:00"
recipe_code = "holdings_with_orders"

## Delete existing data

In [ ]:
# Orders
orders = orders_api.list_orders(filter=f"portfolioId.code eq '{portfolio_code}' and portfolioId.scope eq '{scope}'")

print(f"Removing {len(orders.values)} orders")

for order in orders.values:
    orders_api.delete_order(order.id.scope, order.id.code)

# Allocations
allocations = allocations_api.list_allocations(filter=f"portfolioId.code eq '{portfolio_code}' and portfolioId.scope eq '{scope}'")

print(f"Removing {len(allocations.values)} allocations")

for allocation in allocations.values:
    allocations_api.delete_allocation(allocation.id.scope, allocation.id.code)

# Portfolio
print(f"Removing portfolio {scope}/{portfolio_code}")
try:
    portfolios_api.delete_portfolio(scope, code=portfolio_code)
except:
    print(f'Portfolio not found')

## Create Portfolio

In [ ]:
response = transaction_portfolios_api.create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
        display_name="Live Holdings",
        code=portfolio_code,
        created="2020-01-01T00:00:00+00:00",
        base_currency="USD",
    )
)

print(f"Created portfolio {scope}/{portfolio_code}")

## Load an Instrument Master

In [ ]:
instr_df = pd.read_csv("data/live-orders/instruments.csv")
display(instr_df)

In [ ]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"]
)

succ, failed, errors = cocoon.format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

## Create a recipe

In [ ]:
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        )
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

print("Created recipe")

## Set initial holdings in the parent portfolio

In [ ]:
hldgs_df = pd.read_csv("data/live-orders/holdings.csv")
display(hldgs_df)

In [ ]:
holdings_mapping = {
    "required":{
        "code": f"${portfolio_code}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "ClientInternal": "client_internal",
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    file_type="holdings"
)
succ, failed = cocoon.format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

### Upsert Quotes
Load some sample data for our instrument. We're uploading a quote for today, as market orders are converted to holdings by assuming that they're filled at the instant we make the query and take the latest market price for it.

In [ ]:
quotes_df = pd.read_csv("data/live-orders/quotes.csv")
quotes_df['date'] = datetime.today()
quotes_df

In [ ]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "client_internal",
    "metric_value.unit": "$USD",
}

quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = cocoon.load_from_data_frame(
    api_factory = api_factory,
    scope=scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)
succ, failed, errors = cocoon.format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

## Retrieve initial portfolio holdings

In [ ]:
holdings = transaction_portfolios_api.get_holdings_with_orders(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    recipe_id_scope=scope,
    recipe_id_code=recipe_code)

response_df = lusid_response_to_data_frame(holdings, rename_properties=True)
response_df[["instrumentUid", "holdingType", "units", "cost.amount", "cost.currency"]].style.format({"units":"{:20,.0f}","cost.amount": "{:20,.2f}"})

## Create Orders in LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. We have a number of orders for Amazon stock to demonstrate the processing of market, limit, and stop orders. In addtion we have an order for a corporate bond.

In [ ]:
orders_df = pd.read_csv('data/live-orders/orders.csv')
orders_df

In [ ]:
request = []

for index, order in orders_df.iterrows():
    request.append(lm.OrderRequest(
            id=lm.ResourceId(
                scope=scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': order['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=order['state'],
            type=order['type'],
            price=lm.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']
            ),
            limit_price=lm.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )            
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None,
            
            stop_price=lm.CurrencyAndAmount(
                        amount=order['stop_price'],
                        currency=order['stop_currency']
            )
            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
        )
    )

response = orders_api.upsert_orders(
    order_set_request=lm.OrderSetRequest(
        order_requests=request           
    )
)

lusid_response_to_data_frame(response)

### Post allocations into LUSID
In this section, we post some allocations into LUSID. These allocations represent full or partial allocations against a subset of the originating orders.

In [ ]:
allocations_df = pd.read_csv('data/live-orders/allocations.csv')
allocations_df

In [ ]:
request = []

for index, allocation in allocations_df.iterrows():
    
    # portfolio = allocation['portfolio']

    request.append(lm.AllocationRequest(
            id=lm.ResourceId(
                scope=scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id = lm.ResourceId(
                scope=scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            side=allocation['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': allocation['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=allocation['state'],
            type=allocation['type'],
            price=lm.CurrencyAndAmount(
                amount=allocation['price'],
                currency=allocation['currency']))
    )
    
response = allocations_api.upsert_allocations(
    allocation_set_request=lm.AllocationSetRequest(
        allocation_requests=request
    )
)

lusid_response_to_data_frame(response)

## Check the portfolio holdings with orders
Finally we can ask LUSID to give us a live holdings view that includes contributions from outstanding orders of market, limit, or stop type.

Key:
- P: Position
- O, OC: Order (O) and OrderCash (OC) holdings for the security and cash commitments of each outstanding order. 
- L, LC: Partially and fully-allocated orders will have Allocation (L) and AllocationCash (LC) holdings to represent allocations against them, and have their quantities suitably reduced.

In [ ]:
executed_holdings = transaction_portfolios_api.get_holdings_with_orders(
    scope=scope, 
    code=portfolio_code, 
    property_keys=["Instrument/default/Name"], 
    recipe_id_scope=scope, 
    recipe_id_code=recipe_code)

response_df = lusid_response_to_data_frame(executed_holdings, rename_properties=True)

In [ ]:
effective_date = datetime.now(timezone.utc)

# Setup the aggregation request
aggregation_request = lm.ValuationRequest(
        recipe_id = lm.ResourceId(
            scope = scope,
            code = recipe_code
        ),
        metrics = [
            lm.AggregateSpec(key="Instrument/default/Name", op="Value"),
            lm.AggregateSpec(key="Valuation/PV", op="Proportion"),
            lm.AggregateSpec(key="Valuation/PV", op="Value"),
            lm.AggregateSpec(key="Holding/default/Units", op="Value"),
            lm.AggregateSpec(key="Holding/HoldingType", op="Value"),
        ],
        # choose the valuation date for the request - set using effectiveAt
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_date.isoformat()),
        portfolio_entity_ids = [lm.PortfolioEntityId(
            scope = scope,
            code = portfolio_code,
            portfolio_entity_type="SinglePortfolio")],
        include_order_flow=lm.OrderFlowConfiguration(includeEntityTypes="OrdersAndAllocations")
        )
    
# Pull the data aggregation by passing the effectiveAt date
aggregation = aggregation_api.get_valuation(valuation_request=aggregation_request)
df = pd.DataFrame(aggregation.data)

df

In [ ]:
display(HTML('<h2>Links</h2>'))

display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope={scope}&code={portfolio_code}&entityType=Portfolio&withOrders=true&withAllocations=true" target="_blank">See holdings including open orders</a>'))
display(HTML(f'(If this does not return the data you may need to set the <a href="{api_url}app/system-settings/default-settings/recipe-settings">Default recipe</a> to the recipe "{scope}/{recipe_code}"'))

display(HTML('<br />'))
        
display(HTML(f'<a href="{api_url}app/dashboard/valuations?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&recipeCode={recipe_code}&taxLots=false&includeOrderFlow=OrdersAndAllocations" target="_blank">See valuation including open orders</a>'))